In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras

In [2]:
VOCAB_SIZE = 100
EMBEDDING_SIZE = 10

In [3]:
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

In [4]:
embeddings = keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_SIZE)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

In [5]:
encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

In [6]:
sampler = tfa.seq2seq.sampler.TrainingSampler()

In [7]:
decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(VOCAB_SIZE)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)

In [8]:
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, 
    initial_state=encoder_state, 
    sequence_length=sequence_lengths
)

predictions = tf.nn.softmax(final_outputs.rnn_output)

In [9]:
model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[predictions]
)

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [10]:
x = np.random.randint(100, size=10*1000).reshape(1000, 10)
y = np.random.randint(100, size=15*1000).reshape(1000, 15)
x_decoder = np.c_[np.zeros((1000, 1)), y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([x, x_decoder, seq_lengths], y, epochs=2)

Epoch 1/2
32/32 [==============================] - 11s 336ms/step - loss: 4.6053
Epoch 2/2
32/32 [==============================] - 11s 344ms/step - loss: 4.6031
